In [2]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

c:\Users\Joaquín Amat\Documents\GitHub\skforecast


In [30]:
import re
import pytest
import numpy as np
import pandas as pd
from sklearn.exceptions import NotFittedError
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor

from skforecast.exceptions import IgnoredArgumentWarning
from skforecast.exceptions import UnknownLevelWarning
from skforecast.preprocessing import RollingFeatures
from skforecast.recursive import ForecasterRecursiveMultiSeries


# Fixtures
from skforecast.recursive.tests.tests_forecaster_recursive_multiseries.fixtures_forecaster_recursive_multiseries import series
from skforecast.recursive.tests.tests_forecaster_recursive_multiseries.fixtures_forecaster_recursive_multiseries import exog
from skforecast.recursive.tests.tests_forecaster_recursive_multiseries.fixtures_forecaster_recursive_multiseries import exog_predict

transformer_exog = ColumnTransformer(
                       [('scale', StandardScaler(), ['exog_1']),
                        ('onehot', OneHotEncoder(), ['exog_2'])],
                       remainder = 'passthrough',
                       verbose_feature_names_out = False
                   )

In [39]:
forecaster = ForecasterRecursiveMultiSeries(
    LinearRegression(),
    lags=3, 
    encoding=None
)
forecaster.fit(series=series)

y_true = {
    '1': np.array([1, 2, 3, 4, 5]), 
    '2': np.array([1, 2, 3, 4, 5])
}
y_pred = {
    '1': np.array([0, 0, 0, 0, 0]),
    '2': np.array([0, 0, 0, 0, 0])
}
warn_msg = re.escape(
    ("As `encoding` is set to `None`, no distinction between levels "
        "is made. All residuals are stored in the '_unknown_level' key.")
)
with pytest.warns(UnknownLevelWarning, match = warn_msg):
    forecaster.set_out_sample_residuals(y_true=y_true, y_pred=y_pred)

last_window = pd.DataFrame(forecaster.last_window_)
last_window['3'] = last_window['1']
results = forecaster.predict_bootstrapping(
                steps                   = 1,
                levels                  = ['1', '2', '3'],
                last_window             = last_window,
                n_boot                  = 4,
                use_in_sample_residuals = False
            )

expected = {
    '1': pd.DataFrame(
            data    = np.array([[1.48307367, 2.48307367, 1.48307367, 1.48307367]]),
            columns = [f"pred_boot_{i}" for i in range(4)],
            index   = pd.RangeIndex(50, 51)
        ),
    '2': pd.DataFrame(
            data    = np.array([[5.5061193, 3.5061193, 3.5061193, 2.5061193]]),
            columns = [f"pred_boot_{i}" for i in range(4)],
            index   = pd.RangeIndex(50, 51)
        ),
    '3': pd.DataFrame(
            data    = np.array([[4.48307367, 2.48307367, 4.48307367, 4.48307367]]),
            columns = [f"pred_boot_{i}" for i in range(4)],
            index   = pd.RangeIndex(50, 51)
        )
}

for key in results.keys():
    pd.testing.assert_frame_equal(results[key], expected[key])

c:\Users\Joaquín Amat\Documents\GitHub\skforecast\skforecast\recursive\_forecaster_recursive_multiseries.py:436: UserWarning: When using a linear model, it is recommended to use a transformer_series to ensure all series are in the same scale. You can use, for example, a `StandardScaler` from sklearn.preprocessing.
  warnings.warn(


In [34]:
series.columns

Index(['1', '2'], dtype='object')